In [5]:
!nvidia-smi

Sat Feb 22 12:09:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!pip install ultralytics

In [8]:
!yolo detect train data=african-wildlife.yaml model=yolov8s.pt epochs=30 imgsz=640 batch=8

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=african-wildlife.yaml, epochs=30, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

In [10]:
from ultralytics import YOLO

# Load a model
model = YOLO("/content/runs/detect/train4/weights/best.pt")  # load a brain-tumor fine-tuned model



In [11]:
# Inference using the model
results = model.predict("/content/datasets/african-wildlife/test/images/1 (103).jpg", save=True)


image 1/1 /content/datasets/african-wildlife/test/images/1 (103).jpg: 544x640 1 buffalo, 37.9ms
Speed: 5.4ms preprocess, 37.9ms inference, 157.0ms postprocess per image at shape (1, 3, 544, 640)
Results saved to runs/detect/predict


In [12]:
!pip install yt-dlp opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 61.7 MB/s eta 0:00:00


In [23]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("/content/runs/detect/train4/weights/best.pt")

# Run object tracking on the downloaded YouTube video
results = model.track(source="/content/video.mp4", persist=True, save=True)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1281) /content/video.mp4: 384x640 (no detections), 42.4ms
video 1/1 (frame 2/1281) /content/video.mp4: 384x640 (no detections), 10.7ms
video 1/1 (frame 3/1281) /content/video.mp4: 384x640 (no detections), 10.7ms
video 1/1 (frame 4/1281) /content/video.mp4: 384x640 (no detections), 10.7ms
video 1/1 (frame 5/1281) /content/video.mp4: 384x640 (no detections), 10.7ms
video 1/1 (frame 6/1281) /content/video.mp4: 384x640 (no detections), 1

In [25]:
import numpy as np

# Extract bounding boxes from YOLO tracking
def extract_movement(results):
    movements = []
    for frame in results:
        for obj in frame.boxes.xyxy:
            # Move tensor to CPU and convert to NumPy array
            x1, y1, x2, y2 = obj[:4].cpu().numpy()
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            movements.append((center_x, center_y))
    return np.array(movements)

movement_data = extract_movement(results)


In [26]:
def classify_behavior(movement_data):
    speed = np.linalg.norm(np.diff(movement_data, axis=0), axis=1).mean()

    if speed > 10:  # Adjust threshold based on your dataset
        return "Chased"
    elif np.std(movement_data[:, 0]) < 5:  # Animals moving together
        return "Disaster Response"
    else:
        return "Normal Movement"

behavior_label = classify_behavior(movement_data)
print(f"Predicted Behavior: {behavior_label}")


Predicted Behavior: Chased
